# Logistic Regression From Scratch

Complete implementation including data loading, preprocessing, training, and evaluation.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("logistic_data (1).csv")
df.head()

In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
# Convert categorical/boolean columns using one-hot encoding
for col in X.columns:
    if X[col].dtype == bool:
        X[col] = X[col].astype(int)

X = pd.get_dummies(X, drop_first=True)

X = X.values.astype(float)
y = y.values.astype(float)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
def z_score(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    X_norm = (X - mean) / std
    return X_norm, mean, std

In [ ]:
X_train, x_mean, x_std = z_score(X_train)
X_test = (X_test - x_mean) / x_std

In [ ]:
class LogisticRegression:
    def __init__(self):
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def predict_proba(self, X):
        return self.sigmoid(X @ self.weights + self.bias)

    def predict(self, X, threshold=0.5):
        return (self.predict_proba(X) >= threshold).astype(int)

    def fit(self, X, y, epochs=1000, learning_rate=0.01, threshold=1e-6):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0.0
        prev_loss = float('inf')

        for _ in range(epochs):
            y_pred = self.predict_proba(X)
            loss = -np.mean(
                y * np.log(y_pred + 1e-9) +
                (1 - y) * np.log(1 - y_pred + 1e-9)
            )
            dw = (1 / n_samples) * (X.T @ (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)
            self.weights -= learning_rate * dw
            self.bias -= learning_rate * db
            if abs(prev_loss - loss) < threshold:
                break
            prev_loss = loss

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print('Accuracy:', accuracy)

In [ ]:
indices = np.arange(len(y_test))
plt.figure(figsize=(10,6))
plt.plot(indices, y_test, label='True Labels', marker='o')
plt.plot(indices, y_pred, label='Predicted Labels', marker='x')
plt.xlabel('Data Points')
plt.ylabel('Class')
plt.title('True vs Predicted Labels')
plt.legend()
plt.grid(True)
plt.show()